In [ ]:
%matplotlib notebook
import numpy as np
import scipy.signal
import scipy.fft as sfft
import matplotlib.pylab as plt
from matplotlib import animation

from IPython.display import YouTubeVideo, HTML, Audio
from bokeh.layouts import column, row
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, show, output_notebook
output_notebook()

# Estimadores adaptivos parte I



Hasta ahora hemos estudiando sistemas lineales donde: 
- sus coeficientes quedan fijos luego del diseño y son constantes en el tiempo
- hacen supuestos sobre los estadísticos de la señal/ruido

Qué hacer si
- no podemos hacer supuestos sobre los estadísticos
- los estadísticos de la señal/ruido cambian en el tiempo (no estacionaridad)
- estamos en un escenario donde los datos llegan continuamente (streaming)

Estimador **adaptivo**: 

- Sistemas cuyos coeficientes se pueden adaptar a medida que llegan nuevos datos
- Se diseñan de acuerdo a un método de optimización *online*



<img src="../images/adaptive-systems1.png" width="700">

## Gradiente descendente


- Sea un vector de pesos $w$ de largo $L+1$ que guarda los coeficientes de un filtro
- Sea ahora una función de costo que mapea el vector de pesos a un número real: $J(w): \mathbb{R}^{L+1} \to \mathbb{R}$
    - A menor $J$ mejor es nuestro filtro (menor error)

Para entrenar un filtro adaptivo 
1. Partimos de una solución inicial $w_0$
1. Modificamos iterativamente $w$ tal que $J(w_{t+1}) < J(w_t)$
1. Nos detenemos al cumplir un cierto criterio 

***
Una alternativa de bajo costo para lograr esto es la regla del **gradiente descendente** (GD)

$$
w_{t+1} = w_t - \mu \frac{dJ(w)}{dw},
$$

donde $\mu$ se conoce como tasa de aprendizaje o "paso"


In [ ]:
plt.close('all'); fig, ax = plt.subplots(1, figsize=(7, 3))
x = np.linspace(-4, 6, num=100)
L = lambda x : (x-1)**2 + 3*np.sin(np.pi*x)
p = 10*np.random.rand(10) - 4.
ax.plot(x, L(x))
sc = ax.scatter(p, L(p), s=100)
mu = 0.01

def update(n):
    p = sc.get_offsets()[:, 0]
    p = p - mu*2*(p-1) - mu*3*np.pi*np.cos(np.pi*p)
    sc.set_offsets(np.c_[p, L(p)])
    
anim = animation.FuncAnimation(fig, update, frames=100, interval=200, repeat=False, blit=True)
#plt.close(); HTML(anim.to_html5_video())

***

- Imaginemos $J$ como una superficie de $L+1$ dimensiones
- En cada punto el gradiente negativo de $J$ nos indica hacia donde está el descenso más abrupto
- La tasa $\mu$ nos da el largo del salto entre $w_t$ y $w_{t+1}$

Notemos de la **expansión de Taylor de primer orden** de $J$ en $w_{t}$ que

$$
\begin{align}
J(w_{t+1}) &= J(w_t) + \frac{dJ(w_t)}{dw} (w_{t+1} - w_{t})  \nonumber \\
&= J(w_t) -\mu \left \| \frac{dJ(w_t)}{dw} \right \|^2 \leq J(w_t) \nonumber 
\end{align}
$$

es decir que dado usando la regla GD con $\mu>0$ se cumple que $J$ decrece monotonicamente

- Relación con método de Newton!

### Gradiente descendente en el filtro de Wiener

Para el filtro de Wiener teniamos
$$
J(h) = \sigma_d^2 - 2 \textbf{h}^T R_{ud} + \textbf{h}^T R_{uu} \textbf{h},
$$
por ende
$$
\frac{dJ(h)}{dh} = -2 R_{ud} + 2 R_{uu} \textbf{h}
$$
y finalmente
$$
\textbf{h}_{t+1} = \textbf{h}_{t} (I - 2 \mu R_{uu}) + 2\mu R_{ud}
$$
En este caso la condición de convergencia estable es 
$$
0 < \mu < \frac{1}{\lambda_{\text{max}}},
$$
donde $\lambda_{\text{max}}$ es valor propio más grande de $R_{uu}$

Esto último viene de formar una ecuación de diferencia del estilo $\hat w_{k, t+1} = (1-\mu \lambda_k)^t \hat w_{k, t=0}$

Ref: Haykin, "Adaptive filter theory", 4.3

## Gradiente descendente estocástico (SGD)

El filtro de Wiener es óptimo pero no adaptivo
- Requiere de $N$ muestras de $u$ y $d$ para estimar $R_{ud}$ y $R_{uu}$
- Asume estacionaridad: $J(h) = \mathbb{E}\left[e_n^2\right]$
- El gradiente descendente (GD) es un método deterministico
- Los pesos se adaptan luego de haber presentado las $N$ muestras (batch)

Consideremos el caso en que los datos no son estacionarios
- Significa que debemos adaptar el filtro en cada paso a medida que nuevas muestras son observadas
- Para esto usamos la versión estocástica del GD: SGD
- Los pesos se adaptan luego de haber presentado una muestra o un conjunto pequeño (mini-batch)
- No hay garantía de llegar al óptimo en un problema convexo, pero es más eficiente computacionalmente que GD

<img src="../images/adaptive-sgd.png" width="600">

## Algoritmo Least Mean Square (LMS)


Podemos extender el filtro de Wiener al caso no-estacionario usando SGD

- El resultado es un algoritmo es simple (filtro FIR) que además es robusto
- A diferencia del filtro de Wiener no se requiere conocimiento estadístico del proceso
- Tampoco se requiere calcular e invertir la matriz de correlación
- Se entrena de manera recursiva y online

Consideremos la función de costo estocástica para la arquitectura FIR

$$
\begin{align}
J^s_n(\textbf{w}) &= e_n^2 \nonumber \\
&= (d_n - y_n)^2 \nonumber \\
&= (d_n - \textbf{w}^T \textbf{u}_n )^2 \nonumber \\
&= (d_n - \sum_{k=0}^{L} w_{n, k} u_{n-k} )^2 \nonumber 
\end{align}
$$
donde definimos $\textbf{u}_n = [u_n, u_{n-1}, \ldots, u_{n-L}]$ 

Notemos que usamos el error cuadrático instantaneo en lugar del MSE (filtro de Wiener)

El gradiente en función del peso $w_{n, k}$ es 
$$
\frac{d J^s_n (\textbf{w})}{d w_{n, k}} = - 2 e_n u_{n-k}
$$
Usando la regla SGD llegamos a 
$$
w_{n+1, k} = w_{n, k} + 2 \mu e_n u_{n-k}, k=0, 1, \ldots, L
$$
o en forma matricial
$$
\begin{align}
\textbf{w}_{n+1} &= \textbf{w}_{n} + 2 \mu e_n \textbf{u}_{n}\nonumber \\
&= \textbf{w}_{n} + 2 \mu (d_n -  \textbf{w}_{n}^T \textbf{u}_{n}) \textbf{u}_{n}, \nonumber 
\end{align}
$$

- Estimamos la matriz de correlación instantanea y actualizamos los pesos recursivamente
- La complejidad de este algoritmo es $L+1$, es decir la complejidad del filtro
- Esto se conoce como algoritmo LMS o regla Widrow-Hoff
- Inventado en 1960 por [Bernard Widrow](https://en.wikipedia.org/wiki/Bernard_Widrow) y Ted Hoff

### Interpretación geométrica del algoritmo LMS

Tenemos la siguiente regla iterativa
$$
\textbf{w}_{n+1} = \textbf{w}_{n} + 2 \mu e_n \textbf{u}_{n} = \textbf{w}_{n} + \Delta \textbf{w}_n
$$
que se puede interpretar graficamente como

<img src="../images/adaptive-lms-geometry.png" width="400">

Notemos que

- Los cambios en el vector de peso $\Delta \textbf{w}_n$ son paralelos a $\textbf{u}_{n}$
- Estos cambios podrían estar dominados por $\max_k \textbf{u}_{n} = [u_n, u_{n-1}, \ldots, u_{n-L}]$
- El algoritmo Normalized LMS (NLMS) corrige esto ponderando por la varianza de $\textbf{u}_{n}$ 
$$
\textbf{w}_{n+1} = \textbf{w}_{n} + 2 \mu e_n \frac{\textbf{u}_{n}}{\left(\|\textbf{u}_{n}\|^2 + \delta\right)}
$$
donde la constante $\delta$ es un valor pequeño que se usa para evitar divisiones por cero




### LMS: Adaptive line enhancement (ALE)

- Sistema adaptivo para eliminar ruido aditivo de un canal
- El sistema aprende un filtro pasabanda en torno a la frecuencia de interés
- Notece como es posible filtrar incluso ante cambios bruscos en la señal 

In [ ]:
from numpy.lib.stride_tricks import as_strided
fig, ax = plt.subplots(3, figsize=(9, 5))
t, dt = np.linspace(0, 5, num=500, retstep=True)
u = np.sin(2.0*np.pi*t*5)
#u[250:] += 5  # Cambio abrupto en la media
#u += 2*t  #  Tendencia lineal
#u += u*(0.5 + 0.5*np.cos(2.0*np.pi*t/2))  # Tremolo (AM)
def update(mu, L, rseed):
    np.random.seed(rseed)
    u_noise = u + 0.5*np.random.randn(len(t)) 
    w = np.zeros(shape=(L+1, ))
    ax[0].cla(); ax[1].cla(); ax[2].cla(); 
    #LMS
    u_pred = np.zeros(shape=(len(u), ))
    for k in range(L+1, len(u)-1):
        norm = np.sum(u_noise[k-L-1:k]**2) + 1e-6
        u_pred[k] = np.dot(w, u_noise[k-L-1:k])
        w += 2*mu*(u_noise[k] - u_pred[k])*u_noise[k-L-1:k]/norm
    u_pred[k+1] = np.dot(w, u_noise[k-L-1:k])
    ax[0].plot(t, u_noise, 'k.', alpha=0.5); 
    ax[0].plot(t, u, 'g-', alpha=0.5);  ax[0].plot(t, u_pred); 
    
    ax[1].plot((u - u_pred)**2, label='LMS')
    k, Hk = scipy.signal.freqz(b=w, a=1)
    ax[2].plot(k/(2*dt*np.pi), np.abs(Hk))
    
interact(update, mu=SelectionSlider_nice(options=[1e-4, 1e-3, 0.01, 0.1, 0.2, 1.]),
         L=SelectionSlider_nice(options=[1, 5, 10, 20, 50], value=10), rseed=IntSlider_nice());

- El algoritmo LMS es un sistema de control con retroalimentación
- En el ejemplo anterior notamos la desestabilidad que ocurre con ciertos valores de $\mu$
- La convergencia del algoritmo depende de $\mu$
    - Muy pequeño: Convergencia lenta
    - Muy grande: Desestabilidad

### Comparación entre Filtro de Wiener/GD y algoritmo LMS/SGD
- Wiener: Ambiente estacionario lo cual nos permite calcular $R_{uu}$ y $R_{ud}$. El aprendizaje es determinista.
- LMS: El aprendizaje viene promediando a nivel de los estimadores de $w$. Esta sujeto al ruido de los estimadores de gradiente. El aprendizaje es estadístico.
- Wiener es óptimo en cambio LMS es sub-óptimo (localmente óptimo). LMS tiende a la solución de Wiener
- LMS se actualiza online y tiene costo $L$. Wiener se entrena offline y tiene costo $L^2$

<img src="../images/adaptive-lms.png">


Convergencia del algoritmo LMS (Haykin 6.5)
- El algoritmo LMS tiende en la media $\mathbb{E}[\textbf{w}_n] \to \textbf{w}^*$ para $n\to \infty$
- Convergencia en la media cuadrada: La varianza de $\textbf{w}_n - \textbf{w}^*$ tiene al valor mínimo de $J$ para $n\to \infty$
- Esto se cumple si 
$$
0 < \mu < \frac{1}{\lambda_{\text{max}}}
$$
donde $\lambda_{\text{max}}$ es el valor propio más grande de $R_{uu}$


### LMS: Cancelación de eco

- Supongamos que enviamos una señal de voz a un amig@ 
- Nuestro amig@ escucha lo que enviamos en un parlante y responde
- Nosotros escuchamos la respuesta de nuestro amig@ y adicionalmente nuestro mensaje original

<img src="../images/adaptive-echo-canceller.png" width="500">

- Podemos usar un filtro adaptivo para cancelar el eco
- Usamos como entrada la señal enviada y como salida deseada la señal recibida (con eco)
- El filtro aprende el sistema reverberante
- El error es la nueva señal recibida limpia

In [ ]:
# la señal enviada original
r, fs = sf.read("data/hola1.ogg")
Audio(r, rate=int(fs))

In [ ]:
# El sistema que introduce ecos, por ejemplo una sala
h = np.concatenate(([1.0], np.zeros(10), [0.7], np.zeros(10), [0.5], 
                    np.zeros(10), [0.3], np.zeros(10), [0.1], np.zeros(10), [0.05]))
# la señal enviada con eco
rh = np.convolve(r, h)[:len(r)]
# la señal recibida limpia 
s, fs = sf.read("data/hola2.ogg")
s = np.concatenate((s, np.zeros(shape=(len(r)-len(s)))))
# la señal recibida + señal enviada con eco + ruido blanco
srh = s + 0.3*rh + np.random.randn(len(s))*0.005
Audio(srh, rate=int(fs))

In [ ]:
L, mu = 100, 0.02
fig, ax = plt.subplots(2, figsize=(9, 4))
w = np.zeros(shape=(L+1, ))
rhhat = np.zeros(shape=(len(srh), ))
for k in range(L+1, len(srh)-1):
    norm = np.sum(r[k-L-1:k]**2) + 1e-10
    rhhat[k] = np.dot(w, r[k-L-1:k])
    w += 2*mu*(srh[k] - rhhat[k])*r[k-L-1:k]/(norm)
rhhat[k+1] = np.dot(w, r[k-L-1:k])
shat = srh - rhhat
ax[0].plot(srh, alpha=0.5, label='hola2+hola1');
ax[0].plot(shat, alpha=0.75, label='error');
ax[0].plot(s, alpha=0.75, label='hola2 puro');
ax[0].legend()
ax[1].plot((s - shat)**2)
Audio(shat, rate=int(fs))
